In [23]:
"""
Seagrass Segmentation (Drive Tiles) — Google Colab
===================================================
This notebook trains a **neural-network (U‑Net) classifier** using the training
patches you showed in Drive:


My Drive / GEOG 761_proj_training /
├─ composites/ # yearly summer composites (multi-band .tif)
├─ RGB Composites/ # RGB-only composites (optional)
└─ training_patches_32tile/
├─ images/ # input image tiles (*.tif)
└─ labels/ # label tiles (*.tif) with class ids


It then runs **sliding‑window inference** on any composite GeoTIFF and writes a
classified raster (GeoTIFF) with class ids.


Class codes (expected in label tiles):
0 = background (ignored during training)
1 = sparse seagrass
2 = dense seagrass
3 = exposed sediments
4 = water
5 = interfaces (water–sediment)
"""

'\nSeagrass Segmentation (Drive Tiles) — Google Colab\n===================================================\nThis notebook trains a **neural-network (U‑Net) classifier** using the training\npatches you showed in Drive:\n\n\nMy Drive / GEOG 761_proj_training /\n├─ composites/ # yearly summer composites (multi-band .tif)\n├─ RGB Composites/ # RGB-only composites (optional)\n└─ training_patches_32tile/\n├─ images/ # input image tiles (*.tif)\n└─ labels/ # label tiles (*.tif) with class ids\n\n\nIt then runs **sliding‑window inference** on any composite GeoTIFF and writes a\nclassified raster (GeoTIFF) with class ids.\n\n\nClass codes (expected in label tiles):\n0 = background (ignored during training)\n1 = sparse seagrass\n2 = dense seagrass\n3 = exposed sediments\n4 = water\n5 = interfaces (water–sediment)\n'

In [24]:
#!pip install earthengine-api geopandas shapely rasterio numpy pandas scikit-learn tensorflow==2.15 tqdm

In [25]:
#!pip install rasterio

In [26]:
import geemap
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans

import os, glob, json, math
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.enums import Resampling
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from google.colab import drive

In [27]:
# ================================
# 1) MOUNT DRIVE & PATHS
# ================================


drive.mount('/content/drive', force_remount=True)


ROOT = '/content/drive/My Drive/GEOG 761_proj_training'
IM_DIR = f'{ROOT}/training_patches_32tile/images'
LB_DIR = f'{ROOT}/training_patches_32tile/labels'
COMPOSITES_DIR = f'{ROOT}/composites'
OUT_PRED = f'{ROOT}/predictions' # will be created
os.makedirs(OUT_PRED, exist_ok=True)

Mounted at /content/drive


In [28]:
# ================================
# 2) DISCOVER TRAINING TILES (images ⇄ labels)
# ================================


img_files = sorted(glob.glob(f'{IM_DIR}/*.tif'))
# Match by basename existing in labels
pairs = []
for img in img_files:
  base = os.path.basename(img)
  lb = os.path.join(LB_DIR, base)
  if os.path.exists(lb):
    pairs.append((img, lb))


assert len(pairs) > 0, 'No matching image/label tile pairs found.'
print(f'Found {len(pairs)} tile pairs.')


# Peek first tile to infer shapes and band count
with rasterio.open(pairs[0][0]) as s:
  H, W = s.height, s.width
  C = s.count
print(f'Tile size: {H}x{W}, bands: {C}')

Found 71 tile pairs.
Tile size: 64x64, bands: 4


In [29]:
# ================================
# 3) LOAD TILES INTO MEMORY (small dataset friendly)
# ================================


X_list, y_list = [], []
for i, (im, lb) in enumerate(pairs):
  with rasterio.open(im) as src:
    arr = src.read() # (C,H,W)
  with rasterio.open(lb) as src:
    lab = src.read(1) # (H,W) integer class ids (0..5)
  X_list.append(arr.transpose(1,2,0).astype('float32')) # (H,W,C)
  y_list.append(lab.astype('int32'))


X = np.stack(X_list, axis=0) # (N,H,W,C)
y = np.stack(y_list, axis=0) # (N,H,W)
print('Data shapes:', X.shape, y.shape)

Data shapes: (71, 64, 64, 4) (71, 64, 64)


In [30]:
# ================================
# 4) NORMALIZE & SPLIT
# ================================


# Per‑channel standardization using train split only
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
mean = X_train.mean(axis=(0,1,2), keepdims=True)
std = X_train.std(axis=(0,1,2), keepdims=True) + 1e-6
X_train_n = (X_train - mean) / std
X_val_n = (X_val - mean) / std


# Save scaler for inference
os.makedirs('/content/artefacts', exist_ok=True)
with open('/content/artefacts/scaler.json','w') as f:
  json.dump({'mean': mean.squeeze().tolist(), 'std': std.squeeze().tolist()}, f)


# Optional: compute class weights (ignore 0 during weighting)
classes_present = np.unique(y_train)
classes_present = classes_present[classes_present>0]
cls_w = compute_class_weight('balanced', classes=classes_present, y=y_train[y_train>0])
class_weight = {int(c): float(w) for c, w in zip(classes_present, cls_w)}
print('Class weights (excluding 0):', class_weight)


# Build pixel‑wise weights to **ignore background (0)**
def make_sample_weights(y_batch):
  # weight 0 for background pixels; others from class_weight
  w = np.zeros_like(y_batch, dtype='float32')
  for c, cw in class_weight.items():
    w[y_batch==c] = cw
  return w


sw_train = make_sample_weights(y_train)
sw_val = make_sample_weights(y_val)

Class weights (excluding 0): {1: 7.539086294416244, 2: 6.360599571734475, 3: 0.48887425938117185, 4: 0.4822077922077922, 5: 1.6925356125356126}


In [36]:
# ================================
# 5) MODEL — light U‑Net
# ================================


def conv_block(x, f):
  x = tf.keras.layers.Conv2D(f, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.Conv2D(f, 3, padding='same', activation='relu')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  return x


def encoder_block(x, f):
  c = conv_block(x, f)
  p = tf.keras.layers.MaxPool2D()(c)
  return c, p


def decoder_block(x, skip, f):
  x = tf.keras.layers.Conv2DTranspose(f, 2, strides=2, padding='same')(x)
  x = tf.keras.layers.Concatenate()([x, skip])
  x = conv_block(x, f)
  return x


#inputs = tf.keras.Input(shape=(H, W, C))
inputs = tf.keras.Input(shape=(None, None, C))


s1, p1 = encoder_block(inputs, 32)
s2, p2 = encoder_block(p1, 64)
s3, p3 = encoder_block(p2, 128)


bottleneck = conv_block(p3, 256)


d1 = decoder_block(bottleneck, s3, 128)
d2 = decoder_block(d1, s2, 64)
d3 = decoder_block(d2, s1, 32)


outputs = tf.keras.layers.Conv2D(6, 1, activation='softmax')(d3) # classes: 0..5
model = tf.keras.Model(inputs, outputs)
model.summary()


# Loss that supports per‑pixel sample weights
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy'])


callbacks = [
  tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
]


history = model.fit(
  X_train_n, y_train[..., None],
  validation_data=(X_val_n, y_val[..., None], sw_val), # weights for val reported
  sample_weight=sw_train,
  epochs=80,
  batch_size=8,
  callbacks=callbacks,
  verbose=1)


model.save('/content/artefacts/seagrass_unet.keras')
print('Saved model to /content/artefacts/seagrass_unet.keras')

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None,      │          0 │ -                 │
│ (InputLayer)        │ None, 4)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_30 (Conv2D)  │ (None, None,      │      1,184 │ input_layer_2[0]… │
│                     │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        128 │ conv2d_30[0][0]   │
│ (BatchNormalizatio… │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_31 (Conv2D)  │ (None, None,      │      9,248 │ batch_normalizat… │
│                     │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        128 │ conv2d_31[0][0]   │
│ (BatchNormalizatio… │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, None,      │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ None, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, None,      │     18,496 │ max_pooling2d_6[… │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        256 │ conv2d_32[0][0]   │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, None,      │     36,928 │ batch_normalizat… │
│                     │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        256 │ conv2d_33[0][0]   │
│ (BatchNormalizatio… │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, None,      │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ None, 64)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, None,      │     73,856 │ max_pooling2d_7[… │
│                     │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        512 │ conv2d_34[0][0]   │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_35 (Conv2D)  │ (None, None,      │    147,584 │ batch_normalizat… │
│                     │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, None,      │        512 │ conv2d_35[0][0]   │
│ (BatchNormalizatio… │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, None,      │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ None, 128)        │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_36 (Conv2D)  │ (None, None,      │    295,168 │ max_pooling2d_8[

 Total params: 1,931,686 (7.37 MB)

 Trainable params: 1,928,870 (7.36 MB)

 Non-trainable params: 2,816 (11.00 KB)

Epoch 1/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 944ms/step - accuracy: 0.9413 - loss: nan - val_accuracy: 0.9477 - val_loss: nan
Epoch 2/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9295 - loss: nan - val_accuracy: 0.9477 - val_loss: nan
Epoch 3/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 747ms/step - accuracy: 0.9423 - loss: nan - val_accuracy: 0.9477 - val_loss: nan
Epoch 4/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.9249 - loss: nan - val_accuracy: 0.9477 - val_loss: nan
Epoch 5/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 746ms/step - accuracy: 0.9422 - loss: nan - val_accuracy: 0.9477 - val_loss: nan
Epoch 6/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 793ms/step - accuracy: 0.9384 - loss: nan - val_accuracy: 0.9477 - val_loss: nan
Epoch 7/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.9441 - loss: nan - val_accuracy: 0.9477 - val_loss: nan
Epoch 8/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 987ms/step - accuracy: 0.9350 - loss: nan - val_accuracy: 0.9477 - val_loss: nan
Epoch 9/80
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 786ms/s

In [39]:
# ================================
# 6) INFERENCE — classify ALL composite GeoTIFFs
# ================================

composites = sorted(glob.glob(f'{COMPOSITES_DIR}/*.tif'))
assert len(composites) > 0, 'No composite GeoTIFFs found.'
print('Found composites:', [os.path.basename(p) for p in composites])

# Load scaler
with open('/content/artefacts/scaler.json','r') as f:
    sc = json.load(f)
mu = np.array(sc['mean'], dtype='float32')
sigma = np.array(sc['std'], dtype='float32')

WIN = 256  # multiple of 8

for TARGET in composites:
    print('Classifying:', os.path.basename(TARGET))

    # open source explicitly to keep it alive while writing
    src = rasterio.open(TARGET, 'r')
    try:
        profile = src.profile
        bands_total = src.count
        Hc, Wc = src.height, src.width

        # Use first C bands; error if fewer than C
        if bands_total < C:
            raise ValueError(
                f'{os.path.basename(TARGET)} has {bands_total} bands; model trained on {C}.'
            )
        band_indexes = list(range(1, C + 1))

        out_path = os.path.join(
            OUT_PRED,
            os.path.basename(TARGET).replace('.tif', '_PRED.tif')
        )
        out_profile = profile.copy()
        out_profile.update({'count': 1, 'dtype': 'uint8', 'nodata': 0})

        with rasterio.open(out_path, 'w', **out_profile) as dst:
            for r0 in range(0, Hc, WIN):
                for c0 in range(0, Wc, WIN):
                    h = min(WIN, Hc - r0)
                    w = min(WIN, Wc - c0)
                    win = Window(c0, r0, w, h)

                    block = src.read(indexes=band_indexes, window=win)  # (C,h,w)
                    x = block.transpose(1, 2, 0)[None, ...].astype('float32')

                    # pad edge windows
                    if h < WIN or w < WIN:
                        x_pad = np.zeros((1, WIN, WIN, C), dtype='float32')
                        x_pad[:, :h, :w, :] = x
                        x = x_pad

                    x = (x - mu) / sigma
                    p = model.predict(x, verbose=0)[0]
                    pred = np.argmax(p, axis=-1).astype('uint8')[:h, :w]

                    dst.write(pred, 1, window=win)

        print('Saved:', out_path)
    finally:
        src.close()

print('Legend: 0=background, 1=sparse, 2=dense, 3=sediments, 4=water, 5=interfaces')


Found composites: ['00_Landsat7_Composite_C2_1999_2000_summer.tif', '01_Landsat7_Composite_C2_2000_2001_summer.tif', '02_Landsat7_Composite_C2_2001_2002_summer.tif', '03_Landsat7_Composite_C2_2002_2003_summer.tif', '04_Landsat5_Composite_C2_2003_2004_summer.tif', '05_Landsat5_Composite_C2_2004_2005_summer.tif', '06_Landsat5_Composite_C2_2005_2006_summer.tif', '07_Landsat5_Composite_C2_2006_2007_summer.tif', '08_Landsat5_Composite_C2_2007_2008_summer.tif', '09_Landsat5_Composite_C2_2008_2009_summer.tif', '10_Landsat5_Composite_C2_2009_2010_summer.tif', '11_Landsat5_Composite_C2_2010_2011_summer.tif', '12_Landsat5_Composite_C2_2011_2012_summer.tif', '14_L8_Composite_2014_2015.tif', '15_L8_Composite_2015_2016.tif', '16_L8_Composite_2016_2017.tif', '17_L8_Composite_2017_2018.tif', '18_L8_Composite_2018_2019.tif', '19_L8_Composite_2019_2020.tif', '20_L8_Composite_2020_2021.tif', '21_L8_Composite_2021_2022.tif', '22_L8_Composite_2022_2023.tif', '23_L8_Composite_2023_2024.tif']
Classifying: 0